In [100]:
# Install libraries on first run
#! pip install -q ipynb fastai pathlib import_ipynb

In [101]:
from fastai.tabular.all import *
from pathlib import Path
import import_ipynb
import ipynb
import stockFetcher

In [102]:
testFolder = Path.cwd().parent / 'TestData'
modelFolder = Path.cwd().parent.parent / 'TrainedModels'

In [103]:
print('Model files in modelFolder:')
for file in modelFolder.glob('*.pkl'):
    print(file.name)

Model files in modelFolder:
countryGuesser.pkl
stockScreenerV2.0.pkl
stockScreenerV1.5.pkl
animalClassifier.pkl
stockScreenerV1.4.pkl
stockScreenerV1.6.pkl
leagueChampionClassifier.pkl
stockScreenerV1.7.pkl
politicianClassifier.pkl
stockScreenerV1.3.pkl
stockScreenerV1.2.pkl
stockScreenerV1.0.pkl
stockScreenerV1.1.pkl
test.pkl
stockScreenerV1.9.pkl
stockScreenerV1.8.pkl
stockScreenerV1.10.pkl


In [104]:
evaluations = pd.read_csv(modelFolder / 'modelEvaluations.csv')
bestModel = evaluations.sort_values('MAE', ascending=True).iloc[0]
bestModel.head()

Model Name    stockScreenerV2.0
Timestamp      2025-01-19 12:02
MAE                       0.245
RMSE                        0.3
R2                       -0.582
Name: 11, dtype: object

## Load model

In [105]:
modelName = f"{bestModel['Model Name']}.pkl" # Change this if you want to try other models
learn = load_learner(modelFolder / modelName)

## Predictions

In [106]:
predictionTarget = 'Best' # 'Best' for the best performing tickers 

if predictionTarget == 'Best':
    adr_df = pd.read_csv(testFolder / 'tickers.csv')
    symbols = adr_df['Ticker'].tolist()
    test_df = [stockFetcher.getTickerData(symbol) for symbol in symbols]
    test_df = pd.concat(test_df, ignore_index=True)  # Concatenate list of DataFrames into a single DataFrame

else:
    test_df = stockFetcher.getTickerData(predictionTarget)

# Ensure test_df is a DataFrame
if isinstance(test_df, dict):
    test_df = pd.DataFrame([test_df])

dl = learn.dls.test_dl(test_df)
test_df.head()

$BRK.A: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Error fetching data for BRK.A: single positional indexer is out-of-bounds


/var/folders/ch/yfh31z5s3cq4cw_md59n4qw40000gn/T/ipykernel_81972/1730343049.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_df = pd.concat(test_df, ignore_index=True)  # Concatenate list of DataFrames into a single DataFrame
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)'

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Industry
0,83.199997,85.589996,83.029999,85.120003,21089700,0.0,0.0,-0.045255,2.023515e+11,-17.456805,Internet Retail
1,48.700001,49.840000,48.599998,49.340000,4331600,0.0,0.0,NaN,4.487946e+11,NaN,Internet Content & Information
2,215.169998,215.679993,211.059998,211.500000,21559000,0.0,0.0,-0.256812,1.096955e+12,-3.076181,Semiconductors
3,60.939999,61.790001,60.680000,61.099998,4696900,0.0,0.0,9.970887,7.655341e+10,0.079231,Other Industrial Metals & Mining
4,4.890000,4.930000,4.870000,4.890000,1891200,0.0,0.0,5.237325,7.407176e+10,0.150840,Banks - Diversified


In [109]:
if predictionTarget == 'Best':
    prediction = learn.get_preds(dl=dl)
    sorted_predictions = sorted(zip(symbols, prediction[0]), key=lambda x: x[1], reverse=True)
    print(f"Prediction for best performing tickers:")
    for symbol, pred in sorted_predictions:
        company_name = adr_df[adr_df['Ticker'] == symbol]['Company'].values[0]
        print(f"{symbol} ({company_name}): {pred[0].item() * 100:.2f}%")
else:
    prediction = learn.get_preds(dl=dl)
    company_name = adr_df[adr_df['Ticker'] == predictionTarget]['Company'].values[0]
    print(f"Prediction for {predictionTarget} ({company_name}):")
    print(f"{prediction[0][0][0].item() * 100:.2f}%")

print("Free money?!")

Prediction for best performing tickers:
BIDU (Baidu Inc): 69.13%
JD (JD.com Inc): 51.87%
STM (STMicroelectronics NV): 35.52%
BASFY (BASF SE): 34.78%
DEO (Diageo PLC): 31.25%
BRK.A (Berkshire Hathaway Inc): 27.18%
PKX (POSCO Holdings Inc): 26.68%
CLPBY (China Longyuan Power Group Corp Ltd): 24.55%
XOM (Exxon Mobil Corp): 24.07%
KO (Coca-Cola Co): 23.44%
RELX (RELX PLC): 20.83%
IBN (ICICI Bank Ltd): 20.15%
ABBV (AbbVie Inc): 20.07%
RACE (Ferrari NV): 17.06%
ASML (ASML Holding NV): 17.04%
MFG (Mizuho Financial Group Inc): 15.63%
AZN (AstraZeneca PLC): 15.30%
BNS (Bank of Nova Scotia): 15.03%
V (Visa Inc): 14.30%
TM (Toyota Motor Corp): 13.42%
TCEHY (Tencent Holdings Ltd): 13.22%
SMFG (Sumitomo Mitsui Financial Group Inc): 12.47%
LVMUY (LVMH Moet Hennessy Louis Vuitton SE): 12.44%
HSBC (HSBC Holdings PLC): 11.81%
GSK (GSK PLC): 11.20%
SAN (Banco Santander SA): 10.31%
RIO (Rio Tinto Group): 10.13%
NVS (Novartis AG): 10.06%
BHP (BHP Group Ltd): 9.19%
PHG (Koninklijke Philips NV): 9.17%
SNY (